In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from hypopredict import chunker, labeler
import hypopredict.chunk_preproc as cp
import numpy as np
import pandas as pd
import os

In [3]:
SPLIT = np.array([[83, 64]])
chunk_size = pd.Timedelta('1 hour')
step_size = pd.Timedelta('30 minutes')
ecg_dir = os.getenv('ECG_PATH')
forecast_window = pd.Timedelta('1 hour')
glucose_src = 'local'

In [4]:
glucose_src

'local'

In [5]:
split_chunkified = chunker.chunkify(
                SPLIT.ravel(),
                chunk_size=chunk_size,
                step_size=step_size,
                ecg_dir=ecg_dir,
            )
# label chunks
split_labels = labeler.label_split(
    split_chunkified,
    glucose_src=glucose_src,
    forecast_window=forecast_window,
)
# validate and stack
chunks_split, y_split = cp.filter_and_stack(split_chunkified, split_labels)

Labeling day 83 with 43 chunks
Labeling day 64 with 23 chunks


In [6]:
from hypopredict.cv import CrossValidator
from hypopredict.new_features import prepare_X_y

In [7]:
cv = CrossValidator(SPLIT)

In [8]:
roll_window_size = pd.Timedelta('30 minutes')
roll_step_size = pd.Timedelta('30 minutes')
suffix = 'rolling'
agg_funcs = ['mean']

In [24]:
X_split, y_split = prepare_X_y(
                chunks_split,
                y_split,
                roll_window_size=roll_window_size,
                roll_step_size=roll_step_size,
                suffix=suffix,
                agg_funcs=agg_funcs,
            )  # type: ignore ################ TODO: suffix
            # save for iterative CV

/Users/alexxela/code/hypopredict/hypopredict/new_features.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  chunk = chunks[i]


In [25]:
X_split

,rolling_mean,timestamp,timeofday
0,1982.324649,2014-10-03 23:54:51.424,86091
1,1981.731773,2014-10-03 23:54:51.424,86091
2,1981.731773,2014-10-04 00:24:51.424,1491
3,1981.624847,2014-10-04 00:24:51.424,1491
4,1981.624847,2014-10-04 00:54:51.424,3291
...,...,...,...
1800119,1974.610413,2014-10-04 21:00:26.413,75626
1800120,1974.816720,2014-10-04 21:00:26.413,75626
1800121,1974.816720,2014-10-04 21:30:26.413,77426
1800122,1974.989922,2014-10-04 21:30:26.413,77426


In [27]:
y_split_indexed = pd.Series(y_split, index = X_split['timestamp'])

In [32]:
y_split_indexed

timestamp
2014-10-03 23:54:51.424    0.0
2014-10-03 23:54:51.424    0.0
2014-10-04 00:24:51.424    0.0
2014-10-04 00:24:51.424    0.0
2014-10-04 00:54:51.424    0.0
                          ... 
2014-10-04 21:00:26.413    0.0
2014-10-04 21:00:26.413    0.0
2014-10-04 21:30:26.413    0.0
2014-10-04 21:30:26.413    0.0
2014-10-04 21:57:07.413    0.0
Length: 1800124, dtype: float64

In [34]:
X_split.drop('timestamp', axis =1)

,rolling_mean,timeofday
0,1982.324649,86091
1,1981.731773,86091
2,1981.731773,1491
3,1981.624847,1491
4,1981.624847,3291
...,...,...
1800119,1974.610413,75626
1800120,1974.816720,75626
1800121,1974.816720,77426
1800122,1974.989922,77426


## Check for Demo Days

In [47]:
DEMO_DAYS[0]

83

In [48]:
from hypopredict.params import DEMO_DAYS
from hypopredict.fusion.ml_preproc import MLPreprocessor
import pandas as pd

mlp = MLPreprocessor()
mlp.set_params(n_splits = 1,
               chunk_size = pd.Timedelta('30 minutes'),
               step_size = pd.Timedelta('1 minute'),
               forecast_window = pd.Timedelta('1 hour'),
               roll_window_size = pd.Timedelta('1 minute'),
               roll_step_size = pd.Timedelta('2 minutes')
)
mlp.split([DEMO_DAYS[0]])
mlp.preprocess()

Labeling day 83 with 1274 chunks


/Users/alexxela/code/hypopredict/hypopredict/new_features.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  chunk = chunks[i]


In [50]:
import pickle
with open('83_INDEX.pkl', 'wb') as f:
    pickle.dump(mlp.splits_prepped[0][1].index, f)

In [51]:
DEMO_DAYS[1]

64

In [52]:
from hypopredict.params import DEMO_DAYS
from hypopredict.fusion.ml_preproc import MLPreprocessor
import pandas as pd

mlp = MLPreprocessor()
mlp.set_params(n_splits = 1,
               chunk_size = pd.Timedelta('30 minutes'),
               step_size = pd.Timedelta('1 minute'),
               forecast_window = pd.Timedelta('1 hour'),
               roll_window_size = pd.Timedelta('1 minute'),
               roll_step_size = pd.Timedelta('2 minutes')
)
mlp.split([DEMO_DAYS[1]])
mlp.preprocess()

Labeling day 64 with 687 chunks


/Users/alexxela/code/hypopredict/hypopredict/new_features.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  chunk = chunks[i]


In [53]:
import pickle
with open('64_INDEX.pkl', 'wb') as f:
    pickle.dump(mlp.splits_prepped[0][1].index, f)

In [54]:
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")

filepath = '/Users/alexxela/code/hypopredict/data/processed/fusion_2'
filename = f'64_DEMO_TIME_ml_prepped_cv_splits_{timestamp}.pkl'

dict_to_save = {"splits_prepped": mlp.splits_prepped, "params": mlp.params}

with open(filepath + filename, "wb") as f:
    pickle.dump(dict_to_save, f)